In [5]:
import torch
import torchvision.transforms as transforms
from torchvision import models
from torchvision.models import ResNet50_Weights
from PIL import Image
import os

# Laden des vortrainierten ResNet50-Modells
model = models.resnet50(weights=ResNet50_Weights.DEFAULT)
model.eval()  # Modell im Evaluierungsmodus setzen

# Pfad zur ImageNet-Klassen-Datei
imagenet_classes_path = "imagenet_classes.txt"

# Klassenindex aus ImageNet
imagenet_classes = {idx: entry.strip() for (idx, entry) in enumerate(open(imagenet_classes_path))}

# Transformationen für die Bildvorverarbeitung
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Anpassung der Bildgröße an das Modell
    transforms.ToTensor(),  # Umwandlung in Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalisierung
])

# Verzeichnis mit hochgeladenen Bildern
input_directory = "./uploaded_images"
output_directory = "./filtered_images"
os.makedirs(output_directory, exist_ok=True)

# Funktion zur Klassifikation

def classify_image(image_path):
    try:
        image = Image.open(image_path).convert('RGB')  # Bild öffnen und in RGB konvertieren
        input_tensor = transform(image).unsqueeze(0)  # Transformation und Batch hinzufügen
        with torch.no_grad():
            output = model(input_tensor)  # Vorhersage des Modells
            probabilities = torch.nn.functional.softmax(output[0], dim=0)  # Wahrscheinlichkeiten berechnen
        
        top10_prob, top10_catid = torch.topk(probabilities, 10)  # Top-5-Klassen
        has_snow = False
        has_mountain = False
        for i in range(top10_prob.size(0)):
            class_name = imagenet_classes[top10_catid[i].item()]
            if "snow" in class_name.lower():
                has_snow = True
            if "mountain" in class_name.lower():
                has_mountain = True
        
        # Prüfen, ob sowohl Schnee als auch Berge erkannt wurden
        return has_snow and has_mountain
    except Exception as e:
        print(f"Fehler bei der Verarbeitung von {image_path}: {e}")
    return False

# Bilder filtern und speichern
for filename in os.listdir(input_directory):
    filepath = os.path.join(input_directory, filename)
    if classify_image(filepath):
        output_path = os.path.join(output_directory, filename)
        Image.open(filepath).save(output_path)
        print(f"Bild gespeichert: {output_path}")
    else:
        print(f"Irrelevantes Bild: {filename}")

print("Filterprozess abgeschlossen.")

Bild gespeichert: ./filtered_images\20200209100040_28.jpg
Bild gespeichert: ./filtered_images\20200209100040_29.jpg
Bild gespeichert: ./filtered_images\20200209100040_30.jpg
Bild gespeichert: ./filtered_images\20200209100040_31.jpg
Bild gespeichert: ./filtered_images\20210122090040_23.jpg
Bild gespeichert: ./filtered_images\20210201120002_32.jpg
Irrelevantes Bild: 823947923849927.jpeg
Irrelevantes Bild: ajwiodjiajeidw91832948.jpeg
Bild gespeichert: ./filtered_images\awjdjajwdj3284980.jpeg
Bild gespeichert: ./filtered_images\cam2_20220303_0900-id242_10.jpg
Bild gespeichert: ./filtered_images\cam2_20220303_0900-id242_11.jpg
Bild gespeichert: ./filtered_images\cam2_20220303_0900-id242_7.jpg
Bild gespeichert: ./filtered_images\cam2_20220303_0900-id242_8.jpg
Bild gespeichert: ./filtered_images\cam2_20220303_0900-id242_9.jpg
Bild gespeichert: ./filtered_images\cam2_20220306_1530-id242_42.jpg
Bild gespeichert: ./filtered_images\ekhiejfojqojeo9832905802.jpeg
Bild gespeichert: ./filtered_images